In [ ]:
def compute_depth(centroidx, centroidy, calibrated_left, calibrated_right):
    """
    Computes the depth of the object in the camera frame.
    :param data: centroidx, centroidy, calibrated_left, calibrated_right
    :return: z (depth)
    """